In [1]:
# load all modules
import numpy as np
import pandas as pd
import glob as glob
try: #spelling depends on enviroment version 
    import urllib2 as urllib #URL handling module
except ImportError:
    import urllib.request as urllib
import json

In [2]:
# find all csv files
all_trips = glob.glob("./data/raw/SF/*.csv")
all_trips.sort()

In [3]:
# create a merged list from all csv files
SF_trips = []

for trips_file in all_trips:
    
    # read in trips data
    trips_df = pd.read_csv(trips_file, index_col=None, header=0, skipinitialspace=True, usecols=['start_lat', 'start_lng', 'end_lng', 'end_lat', 'start_station_id',
    'end_station_id'])
    
    # append it to the results list
    SF_trips.append(trips_df)

In [4]:
# create a merged df from all trips
SF_trips_df = pd.concat(SF_trips, axis=0, ignore_index=True) 

In [6]:
# dimensions of dataframe
SF_trips_df.shape 

(1586016, 6)

In [8]:
# find null values
SF_trips_df.isna().sum() 

start_station_id    475234
end_station_id      509286
start_lat                0
start_lng                0
end_lat               1781
end_lng               1781
dtype: int64

In [9]:
# drop null values
SF_trips_df.dropna(inplace=True)
SF_trips_df.shape

(914833, 6)

In [11]:
# Bounding box for San Francisco
lat_min = 37.705262390821154
lat_max = 37.81066405821323
lng_min = -122.5245991704898
lng_max = -122.35292764505348

In [13]:
#sanity check for values equal to zero
SF_trips_df = SF_trips_df[(SF_trips_df['start_lat'] >= lat_min) & (SF_trips_df['start_lat'] <= lat_max) & \
                      (SF_trips_df['start_lng'] >= lng_min) & (SF_trips_df['start_lng'] <= lng_max)]

In [14]:
# shape check after cleaning
SF_trips_df.shape 

(770918, 6)

In [18]:
# change datatype of station id to integer
SF_trips_df.start_station_id = SF_trips_df.start_station_id.astype(str)

# remove duplicate stations
SF_stations_df = SF_trips_df.drop_duplicates(subset = ["start_station_id"])

In [21]:
# shape check after dropping duplicates
SF_stations_df.shape

(260, 6)

In [22]:
# sort dataframe and reset index
SF_stations_df = SF_stations_df.sort_values('start_station_id')
SF_stations_df.reset_index(drop=True, inplace=True)
SF_stations_df

,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng
0,16th Depot Bike Station,16th Depot Bike Station,37.766337,-122.396054,37.766219,-122.396018
1,16th St Depot,SF-L29,37.754798,-122.389680,37.772456,-122.392689
2,SF I29-1,SF-G30-1,37.783172,-122.393572,37.788059,-122.391865
3,SF-A19,SF-H12,37.805230,-122.437259,37.779556,-122.463121
4,SF-A20,SF-H29,37.804272,-122.433537,37.785377,-122.396906
...,...,...,...,...,...,...
255,SF-Y30,SF-Q30,37.729393,-122.386537,37.755367,-122.388795
256,SF-Z16,SF-U21,37.722555,-122.443727,37.739819,-122.425445
257,SF-Z28,SF-Y29,37.725404,-122.394058,37.729253,-122.393090
258,SF-Z6,SF-M11,37.722001,-122.482735,37.764224,-122.467377


In [24]:
'''
Input: the file that contain origin-destination lat/long information;
Output: list of elevation corresponding the input O-D pairs
'''
apiKey = '90Wycb0HEdciPzdZGzI_mXr87SfpxWj0TGVyAAB2apM'
s_elv_list = []

for index, row in SF_stations_df.iterrows():
    start_lat = row['start_lat']
    start_lng = row['start_lng']
    end_lat = row['end_lat']
    end_lng = row['end_lng']
    
    url = str('https://router.hereapi.com/v8/routes?apiKey={}&origin={},{}&transportMode=car&destination={},{}&return=elevation'.format(apiKey, start_lat, start_lng, end_lat, end_lng ))
    data = urllib.urlopen(url).read().decode('utf-8')
    data = json.loads(data)
    
    # append elevation to 'start' list, nan if no elevation retrieved
    try:    
        s_elv = data['routes'][0]['sections'][0]['departure']['place']['location']['elv']
    except:
        s_elv = np.nan
        
    s_elv_list.append(s_elv)

In [25]:
# add start point elevation from list
SF_stations_df['start_station_elv'] = s_elv_list

In [26]:
# make changes to dataframe for csv export
SF_stations_df.drop(['end_station_id', 'end_lat', 'end_lng'], axis = 1, inplace=True)
SF_stations_df.rename(columns={'start_station_id': 'station_id', 'start_lat': 'station_latitude', \
                                        'start_lng': 'station_longitude', 'start_station_elv': 'station_elevation'}, inplace=True)
SF_stations_df

,station_id,station_latitude,station_longitude,station_elevation
0,16th Depot Bike Station,37.766337,-122.396054,0.0
1,16th St Depot,37.754798,-122.389680,-24.0
2,SF I29-1,37.783172,-122.393572,-10.0
3,SF-A19,37.805230,-122.437259,-26.0
4,SF-A20,37.804272,-122.433537,-19.0
...,...,...,...,...
255,SF-Y30,37.729393,-122.386537,-7.0
256,SF-Z16,37.722555,-122.443727,26.0
257,SF-Z28,37.725404,-122.394058,-23.0
258,SF-Z6,37.722001,-122.482735,1.0


In [28]:
# export to csv
SF_stations_df.to_csv('./data/exports/sf_stations_elevation.csv', encoding='utf-8', index=False)